In [1]:
import shutil

import matplotlib.pyplot as plt
import pandas as pd

import evalml

from functions import *

pd.set_option('display.float_format', '{:.6f}'.format)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
2023-10-30 19:18:53.857996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 19:18:54.709716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data

In [2]:
min_non_zero_pct = 0.5
window_size = 3
sequence_length = 10
number_test_sequences = 3

df = load_and_merge()
df = remove_underpopulated_taxa(df, min_non_zero_pct)
df = standard_rolling_average(df, window_size)
df = feature_wise_scaling(df)

In [3]:
df

,659361,364179,3439402,181155,302746,4473763,175535,3887769,585227,189384,...,3304236,584137,193233,4334711,196664,583117,354850,subject_id,sampling_day,ind_time
E000823.4.0,0.000000,0.000160,0.288889,0.000000,0.000000,0.242718,0.010256,0.190476,0.000000,0.100671,...,0.029412,0.000000,0.040201,0.455696,0.059699,0.010154,0.064039,E000823,120,4.000000
E000823.5.0,0.000000,0.000160,0.200000,0.000000,0.000000,0.199029,0.007692,0.119048,0.000000,0.120805,...,0.117647,0.000000,0.035804,0.398734,0.054162,0.010779,0.029557,E000823,150,5.000000
E000823.5.7,0.000000,0.000180,0.200000,0.003724,0.000000,0.228155,0.007692,0.023810,0.000000,0.053691,...,0.117647,0.000000,0.034338,0.518987,0.060045,0.009685,0.019704,E000823,171,5.700000
E000823.6.7,0.065636,0.000160,0.088889,0.016139,0.000000,0.121359,0.002564,0.023810,0.000000,0.080537,...,0.205882,0.000085,0.021566,0.386076,0.035127,0.006092,0.000000,E000823,201,6.700000
E000823.8.5,0.065636,0.000220,0.177778,0.017381,0.000000,0.490291,0.020513,0.071429,0.000000,0.053691,...,0.176471,0.000085,0.050251,0.386076,0.061602,0.037334,0.000000,E000823,255,8.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E014086.30.4,0.060355,0.080890,0.288889,0.007449,0.044379,0.111650,0.025641,0.476190,0.479017,0.503356,...,0.470588,0.055292,0.091290,0.253165,0.127531,0.423113,0.068966,E014086,912,30.400000
E014086.32.4,0.060355,0.120524,0.622222,0.010552,0.077663,0.194175,0.071795,0.404762,0.880928,0.711409,...,0.441176,0.071990,0.142169,0.341772,0.345042,0.622876,0.285714,E014086,972,32.400000
E014086.33.5,0.051679,0.141802,1.000000,0.011173,0.161243,0.436893,0.074359,0.619048,0.849198,0.906040,...,0.264706,0.057024,0.213358,0.373418,0.537809,0.756436,0.507389,E014086,1005,33.500000
E014086.34.4,0.021124,0.127490,0.977778,0.004345,0.160503,0.572816,0.071795,0.571429,0.761174,0.899329,...,0.294118,0.037543,0.241625,0.316456,0.631597,0.749125,0.610837,E014086,1032,34.400000


## Features and targets

In [4]:
train_feats, train_targets, val_feats, val_targets, test_feats, test_targets, test_subjects = xgboost_flat_feats_and_targets(df, n_test_seq=number_test_sequences, seq_length=sequence_length)

The test subjects are ['E006493' 'E013094' 'E003188']


In [5]:
train_feats, _, train_targets, _ = evalml.preprocessing.split_data(train_feats, train_targets, problem_type="regression", test_size=0.001)

In [6]:
train_feats

,0,1,2,3,4,5,6,7,8,9,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
71,0.171633,0.003395,0.002641,0.002263,0.001132,0.004904,0.027914,0.110524,0.107129,0.085251,...,15.800000,16.900000,19.200000,20.000000,21.100000,22.500000,23.500000,24.800000,27.900000,29.000000
37,0.162580,0.166352,0.150132,0.050170,0.144474,0.143342,0.142588,0.027160,0.042625,0.040739,...,22.800000,23.600000,25.600000,26.900000,27.800000,28.800000,29.800000,30.900000,33.000000,34.000000
74,0.291965,0.309694,0.161449,0.133157,0.107129,0.483968,0.887212,1.000000,0.783855,0.377593,...,14.800000,15.700000,16.700000,17.800000,19.000000,19.700000,21.000000,22.700000,24.200000,24.900000
108,0.425500,0.379102,0.000377,0.000377,0.000000,0.002263,0.006413,0.132780,0.160317,0.216899,...,1.900000,3.100000,3.700000,4.900000,5.800000,7.600000,8.000000,8.700000,10.000000,10.600000
228,0.000000,0.000000,0.000000,0.000000,0.048284,0.176160,0.206714,0.271973,0.188608,0.304791,...,7.000000,8.400000,9.100000,10.300000,12.100000,13.100000,13.900000,15.400000,16.400000,17.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.022256,0.022256,0.028668,0.006413,0.007167,0.018106,0.143342,0.262542,0.265183,0.158054,...,7.900000,8.900000,9.700000,10.700000,12.000000,13.800000,15.200000,15.800000,17.000000,18.000000
192,0.013580,0.013580,0.054319,0.042248,0.042248,0.000000,0.000377,0.000377,0.000377,0.000377,...,12.600000,13.500000,14.600000,15.600000,16.600000,17.600000,18.400000,19.400000,20.400000,21.400000
117,0.010939,0.000377,0.000000,0.000000,0.000377,0.000377,0.000377,0.002641,0.004149,0.004149,...,2.600000,3.700000,4.300000,5.700000,7.900000,8.500000,9.100000,10.500000,12.600000,13.600000
47,0.000000,0.068276,0.070162,0.070162,0.001886,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.200000,14.900000,15.800000,18.000000,19.500000,20.600000,21.700000,23.800000,25.500000,28.000000


In order to properly train an XGBoost model we need a proper validation set

## The model

In [7]:
import xgboost as xgb
model = xgb.XGBRegressor(objective='reg:squarederror', 
                         n_estimators=10, 
                         max_leaves=100,
                         max_depth=100,  
                         n_jobs=50)

In [ ]:
model.fit(train_feats, train_targets, eval_set=[(val_feats, val_targets)], verbose=True)

## Validation

In [ ]:
test_subjects

In [ ]:
target_subject_idx = 0
target_subject = test_subjects[target_subject_idx]

subject_feats = test_feats[target_subject]
subject_targets = test_targets[target_subject]

In [ ]:
subject_feats, _, subject_targets, _ = evalml.preprocessing.split_data(subject_feats, subject_targets, problem_type="regression", test_size=0.001)

In [ ]:
subject_targets

In [ ]:
pred_df = pd.DataFrame(model.predict(subject_feats))
pred_df.columns = subject_targets.columns
pred_df

In [ ]:
test_targets[target_subject]

In [ ]:
def fetch_subject_y_true_and_y_pred(test_subjects, model, df, target_subject_idx, test_feats, test_targets):
    
    cols = df.columns
    cols = cols[~cols.isin(ignore_cols)]
    
    target_subject = test_subjects[target_subject_idx]

    test_subject_feats = np.asarray(test_feats[target_subject])
    test_subject_targets_df = test_targets[target_subject]
    
    test_subject_targets_df.columns = cols
    test_subject_y_pred = model.predict(test_subject_feats)
    
    test_subject_y_pred_df = pd.DataFrame(test_subject_y_pred)
    test_subject_y_pred_df.columns = cols
    test_subject_y_pred_df.index = test_subject_targets_df.index

    return test_subject_targets_df, test_subject_y_pred_df

In [ ]:
subjects_error_dfs = []
for target_subject_idx in range(len(test_subjects)):
    
    test_subject_targets_df, test_subject_y_pred_df = fetch_subject_y_true_and_y_pred(test_subjects, model, df, target_subject_idx, test_feats, test_targets)

    errors_df = calculate_percentage_errors(test_subject_y_pred_df, test_subject_targets_df)
    subjects_error_dfs.append(errors_df)
    
errors_df = pd.concat(subjects_error_dfs)

In [ ]:
percentile_graph(errors_df, "accross all taxa XGBOOST", y_top_lim=1)